<a href="https://colab.research.google.com/github/Komalsai234/nl2sql/blob/main/notebooks/nl2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NL2SQL

In [3]:
!pip install python-dotenv >/dev/null;
!pip install langchain >/dev/null;
!pip install langchain-community >/dev/null;
!pip install langchain-openai >/dev/null;
!pip install sqlalchemy psycopg2-binary -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.2 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="env")

True

In [5]:
import os
import json
import logging
import threading
from typing import Dict, Any

from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.sql import text
from langchain.docstore.document import Document

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Data handler

In [6]:
DESIRED_TABLES = [
    "auth_user",
    "sourcing_job",
    "sourcing_applicationstagemovement",
    "param_auth_businessunit",
    "sourcing_candidate",
    "sourcing_applicationstatus",
    "sourcing_application",
    "sourcing_hiringteammember",
    "sourcing_applicationcategory",
    "sourcing_candidatesourcetype",
    "sourcing_resume",
    "consultancy_consultancy",
    "consultancy_consultant",
    "consultancy_outsourcedjob",
    "employee_employee",
    "param_auth_organization",
    "sourcing_jobcategory",
    "offer_manager_offermasterdata",
    "sourcing_jobactivitylog",
    "param_auth_teammember",
    "param_forms_standardapplicationformfield",
    "param_forms_formresponse",
    "param_forms_formshare",
    "param_forms_formreviewer",
    "param_forms_standardapplicationformsection",
    "param_forms_entitycustomfields",
    "param_forms_formsection",
    "param_forms_formtemplate",
    "reports_sourceorigincounts",
    "sourcing_city",
    "sourcing_location",
    "sourcing_jobreviewers",
    "sourcing_recruiter",
    "sourcing_recruiternotes",
    "sourcing_recruiternotes_mentions",
    "sourcing_prescreeningdata",
    "sourcing_referrals",
    "sourcing_tag",
    "param_auth_team",
    "interview_kit_interviewsession",
    "param_forms_formquestion",
    "sourcing_recruiter_jobs"
]

In [7]:
TABLE_DESCRIPTIONS = {
    "auth_user": "Manages user accounts in the HR application, storing usernames, hashed passwords, email addresses, and roles (superuser, staff, active status). Tracks login and account creation timestamps, enabling authentication, authorization, permission management, and secure access to recruitment tools, user assignments, and action tracking.",
    "consultancy_consultancy": "Stores external consultancy details for HR management, including names, creation emails, lock status, and sourcing team affiliations. Tracks creation and modification timestamps, enabling coordination of outsourced recruitment, consultancy relationship management, and monitoring of external hiring contributions.",
    "consultancy_consultant": "Holds profiles of individual consultants with their emails and links to their parent consultancies, along with timestamps for creation and modification. It is essential for identifying and contacting consultants, facilitating collaboration with external recruiters, and is applied in consultancy-driven recruitment to manage consultant interactions.",
    "consultancy_outsourcedjob": "Links outsourced jobs to consultancies, storing job IDs, consultancy IDs, and validity dates to track assignment expirations. Facilitates management of external job postings, consultancy recruitment responsibilities, and alignment with organizational hiring needs.",
    "employee_employee": "Stores employee profiles for HR management, including employee IDs, designations, departments, locations, preferred locations, reporting managers, and business units. Supports JSONB fields for custom data and resume storage, enabling recruitment, onboarding, team organization, workforce analytics, and employee tracking.",
    "interview_kit_interviewsession": "Manages interview sessions in the recruitment pipeline, storing start/end times, locations, recruiter notes, JSONB interviewer feedback, and links to candidates, jobs, and team members. Tracks cancellation status, notifications, and interview URLs, enabling scheduling, candidate progress monitoring, and feedback collection for hiring coordination.",
    "offer_manager_offermasterdata": "Stores job offer details for candidates, including JSONB data with salary components (basic, HRA, LTA, PF, gratuity), job designation, department, location, and recruiter information. Links to candidates and jobs, enabling offer management, compensation tracking, and recruitment finalization.",
    "param_auth_businessunit": "Defines business units with names, ERP codes, and links to teams and organizations, tracking creation and modification timestamps. Supports categorization of jobs and recruitment by business unit, enabling targeted hiring strategies and alignment with organizational structures.",
    "param_auth_organization": "Stores top-level organization details, including names, ERP codes, and team associations, with creation and modification timestamps. Enables management of organizational hierarchies, supports multi-entity recruitment context, and aligns HR operations with team and entity structures.",
    "param_auth_team": "Defines recruitment teams with company names, logos, JSONB configurations, and authentication settings (Okta, SAML, MFA). Includes team types, career page settings, and integration details, enabling team-based HR workflows, branding, and secure recruitment coordination.",
    "param_auth_teammember": "Assigns team members with roles, statuses, and JSONB configurations for onboarding and activities, linking to teams and users. Manages recruitment permissions, tracks member involvement, and supports collaboration and onboarding in HR workflows.",
    "param_forms_entitycustomfields": "Stores custom field values for entities (e.g., jobs, candidates) with text data, entity types, and links to users and teams, tracking creation and modification. Enhances recruitment flexibility by enabling tailored data collection and custom attributes for HR processes",
    "param_forms_formquestion": "Stores form questions with text values, repeatability flags, and field types, linking to jobs, sections, teams, and templates. Enables dynamic application form creation, targeted candidate data collection, and customized recruitment question design.",
    "param_forms_formresponse": "Captures candidate form responses with text values, sequences, and dropdown options, linking to applications, candidates, jobs, questions, and teams. Supports structured evaluation of applicant answers, form submission processing, and recruitment data analysis.",
    "param_forms_formreviewer": "Assigns reviewers to form submissions with statuses (e.g., pending, invited) and comments, linking to form shares, team members, and teams. Facilitates organized candidate evaluation, feedback coordination, and thorough review of application responses in recruitment.",
    "param_forms_formsection": "Organizes form sections with text values, linking to jobs, teams, and templates, with creation and modification timestamps. Structures application forms into logical sections, enhancing data collection and recruitment form layout design.",
    "param_forms_formshare": "Tracks form sharing with candidates, storing JSONB responses, validity dates, and links to candidates, jobs, and creators. Manages form distribution, ensures candidate completion, and facilitates application workflows in recruitment.",
    "param_forms_formtemplate": "Defines reusable form templates with names, JSONB sections, rules, and reviewer settings, linked to teams and creators. Standardizes candidate data collection, streamlines application processes, and supports consistent recruitment form management.",
    "param_forms_standardapplicationformfield": "Specifies application form fields with titles, types (e.g., text, number), and properties (required, editable), linked to sections and teams. Enables customized form design, targeted candidate data capture, and tailored input fields for recruitment.",
    "param_forms_standardapplicationformsection": "Organizes standard form sections with titles, types, and priorities, linked to teams. Groups related fields for consistent application templates, enhancing form usability and recruitment data organization.",
    "reports_sourceorigincounts": "Tracks recruitment source counts in JSONB format (e.g., applicants, interviewed, offered), linked to teams with timestamps. Enables analysis of sourcing effectiveness, channel performance, and recruitment strategy optimization in HR reporting.",
    "sourcing_application": "Links candidates to job postings, storing application details like timestamps, statuses, JSONB form responses, rejection reasons, and assigned recruiters. Manages the application lifecycle, tracks candidate progress, and supports recruitment evaluation and advancement.",
    "sourcing_applicationcategory": "Categorizes applications with titles and priority levels, enabling structured grouping and prioritization. Supports recruitment organization, application management, and trend analysis for streamlined workflows and reporting.",
    "sourcing_applicationstagemovement": "Tracks application stage transitions with timestamps, source and destination stages, and involved team members, linking to applications and statuses. Monitors candidate pipeline progress, ensures hiring process transparency, and supports recruitment stage management.",
    "sourcing_applicationstatus": "Defines application statuses with titles, stages, and category links, tracking creation and modification timestamps. Standardizes candidate progress tracking, supports consistent recruitment evaluation, and enables status updates and reporting.",
    "sourcing_candidate": "Maintains candidate profiles with names, emails, resumes, experience, locations, and JSONB tags, linked to sources, teams, and resume storage. Centralizes data for sourcing, screening, and tracking candidate qualifications throughout recruitment.",
    "sourcing_candidatesourcetype": "Categorizes candidate sources with types, categories, and team links, defining recruitment origins. Tracks sourcing effectiveness, optimizes outreach channels, and supports analysis of candidate recruitment sources.",
    "sourcing_city": "Stores city details with locality names, city names, and autocomplete flags, tracking creation and modification. Enables location-based job and candidate filtering, matching opportunities to preferences, and enhances recruitment search capabilities.",
    "sourcing_hiringteammember": "Assigns team members to jobs with roles, user IDs, and locations, linking to jobs and users. Defines hiring team responsibilities, fosters recruitment collaboration, and manages oversight of job postings and candidate evaluations.",
    "sourcing_job": "Manages job postings with titles, JSONB descriptions, statuses, experience, and locations, linked to teams, categories, and form templates. Supports job creation, tracking, and publication, driving recruitment from posting to closure.",
    "sourcing_jobactivitylog": "Logs job activities with types, actions, and metadata, linking to jobs and users. Maintains a recruitment audit trail, ensures transparency, and tracks job-related changes for monitoring.",
    "sourcing_jobcategory": "Categorizes jobs with names and ERP codes, linked to teams for clarity. Groups jobs by type or function, simplifies recruitment searches and reporting, and aids in filtering opportunities for candidates and HR.",
    "sourcing_jobreviewers": "Tracks job application reviewers with order, status, and comments, linking to jobs, team members, and inviters. Manages the review process, ensures structured feedback, and coordinates evaluations and approvals in recruitment.",
    "sourcing_location": "Stores job location data with unique IDs and names for geographical categorization. Associates jobs with places, supports location-based recruitment searches, and aligns opportunities with candidate or organizational needs.",
    "sourcing_prescreeningdata": "Stores prescreening data with statuses, timestamps, scores, and JSONB inferences, linking to candidates and jobs. Facilitates early candidate evaluation, streamlines screening processes, and supports recruitment filtering before interviews.",
    "sourcing_recruiter": "Stores recruiter profiles with names, emails, and timestamps, identified by IDs and ATS references. Manages recruiter identities, enables candidate communication, and supports assignment and tracking of recruitment responsibilities.",
    "sourcing_recruiter_jobs": "Connects recruiters to jobs with IDs, ensuring clear accountability. Tracks recruiter workloads, manages job assignments, and supports efficient recruitment oversight for specific postings.",
    "sourcing_recruiternotes": "Stores recruiter notes with text, privacy flags, and edit tracking, linking to candidates, jobs, and recruiters. Documents observations, supports candidate evaluation, and enhances recruitment collaboration.",
    "sourcing_recruiternotes_mentions": "Tracks user mentions in recruiter notes, linking to notes and users. Enhances team communication, ensures awareness, and facilitates coordination in recruitment processes.",
    "sourcing_referrals": "Manages candidate referrals with emails, types, and relationships, linked to candidates and teams. Drives referral programs, incentivizes participation, and tracks recommended candidates in recruitment.",
    "sourcing_resume": "Stores resumes with file paths, JSONB-parsed data, skills, and parsing status, tracking errors. Enables access to candidate qualifications, supports credential evaluation, and streamlines recruitment reviews.",
    "sourcing_tag": "Stores tags with titles, colors, and team member links for categorizing recruitment entities. Organizes and filters data, simplifies searches, and enhances tagging of profiles or jobs in recruitment."
}

In [8]:
class DataHandler:
    def __init__(
        self,
        db_uri: str = os.getenv("DATABASE_URI"),
    ):

        if not db_uri:
            raise ValueError("DATABASE_URI not found in environment variables.")
        self.db_uri = db_uri
        self.engine = create_engine(self.db_uri)
        self._last_error = threading.local()

        try:
            with self.engine.connect() as connection:
                logging.info("Successfully connected to the PostgreSQL database.")
        except SQLAlchemyError as e:
            logging.error(f"Failed to connect to the database: {e}")
            raise

    def get_schema(self) -> Dict[str, Any]:
        inspector = inspect(self.engine)
        schema = {}
        for table_name in inspector.get_table_names():
            if table_name not in DESIRED_TABLES:
                continue
            columns = inspector.get_columns(table_name)
            if not columns:
                continue
            schema[table_name] = {
                "description": TABLE_DESCRIPTIONS.get(table_name, ""),
                "columns": {col["name"]: col["type"].__str__() for col in columns},
                "primary_keys": inspector.get_pk_constraint(table_name).get(
                    "constrained_columns", []
                ),
                "foreign_keys": [
                    {
                        "column": fk["constrained_columns"][0],
                        "references": {
                            "table": fk["referred_table"],
                            "column": fk["referred_columns"][0],
                        },
                    }
                    for fk in inspector.get_foreign_keys(table_name)
                    if fk["constrained_columns"] and fk["referred_columns"]
                ],
            }
        return schema

    def _format_schema(self, schema: Dict[str, Any]) -> tuple[list[str], list[str]]:
        docs = []
        table_names = []
        for table, details in schema.items():
            columns_str = ", ".join(
                f"{k}: {v}" for k, v in details.get("columns", {}).items()
            )
            primary_keys_str = ", ".join(details.get("primary_keys", [])) or "None"
            foreign_keys_str = (
                "; ".join(
                    [
                        f"{fk['column']} -> {fk['references']['table']}.{fk['references']['column']}"
                        for fk in details.get("foreign_keys", [])
                    ]
                )
                or "None"
            )
            text = (
                f"Table: {table}\n"
                f"Description: {details.get('description', '')}\n"
                f"Columns: {columns_str}\n"
                f"Primary Keys: {primary_keys_str}\n"
                f"Foreign Keys: {foreign_keys_str}\n"
            )
            doc = Document(page_content=text, metadata={"source": "local"})
            docs.append(doc)
            table_names.append(table)
        return docs, table_names

    def get_formatted_schema(
        self, schema: Dict[str, Any]
    ) -> tuple[list[str], list[str]]:
        return self._format_schema(schema)

In [9]:
handler = DataHandler()

In [ ]:
schema = handler.get_schema()
docs, table_names = handler.get_formatted_schema()

## Selector

In [ ]:
class Selector:
  pass

## Decomposer

In [ ]:
class Decomposer:
  pass

## Refiner

In [ ]:
class Refiner:
  pass